In [1]:
import os
import dotenv

import elasticsearch
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.vectorstores import ElasticsearchStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

dotenv = dotenv.load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")  

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!huggingface-cli login --token {HF_TOKEN}

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/npn/.cache/huggingface/token
Login successful


In [3]:
loader = HuggingFaceDatasetLoader(path='nlplabtdtu/university-dataset', page_content_column='body')
documents = loader.load()

/opt/homebrew/Caskroom/miniforge/base/envs/nlp/lib/python3.9/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(


In [4]:
_documents = documents[:10000]

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
docs = text_splitter.split_documents(_documents)

Created a chunk of size 1549, which is longer than the specified 1500
Created a chunk of size 1833, which is longer than the specified 1500
Created a chunk of size 2861, which is longer than the specified 1500
Created a chunk of size 2099, which is longer than the specified 1500
Created a chunk of size 1752, which is longer than the specified 1500
Created a chunk of size 2996, which is longer than the specified 1500
Created a chunk of size 2142, which is longer than the specified 1500
Created a chunk of size 2036, which is longer than the specified 1500
Created a chunk of size 7251, which is longer than the specified 1500
Created a chunk of size 2652, which is longer than the specified 1500
Created a chunk of size 2904, which is longer than the specified 1500
Created a chunk of size 2902, which is longer than the specified 1500
Created a chunk of size 1608, which is longer than the specified 1500
Created a chunk of size 1652, which is longer than the specified 1500
Created a chunk of s

In [6]:
len(docs)

38698

In [59]:
embeddings = HuggingFaceEmbeddings(model_name='nlplabtdtu/sbert-70M-cased')

db = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    es_url="http://localhost:9200", 
    index_name="education",
)

In [61]:
db.client.indices.refresh(index="education")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [66]:
query = "Đối tượng tuyển thẳng của trường trung cấp công an nhân dân"
results = db.similarity_search(query, k=2)
print(results)

[Document(page_content='– Dự tuyển đào tạo liên thông giữa trình độ trung cấp, trình độ cao đẳng với\ntrình độ đại học;\n\n– Dự tuyển đào tạo đại học cấp bằng đại học thứ hai (văn bằng 2);\n\n– Dự tuyển đào tạo trung cấp chính quy tuyển mới;\n\n– Dự tuyển đào tạo trung cấp chính quy cho cán bộ trong biên chế CAND;\n\n– Dự tuyển giáo dục trung học phổ thông (đào tạo văn hóa nhỏ tuổi);\n\n– Dự tuyển đào tạo cao cấp lý luận chính trị và hoàn chỉnh kiến thức cao cấp\nlý luận chính trị;\n\n– Dự tuyển hoàn thiện kiến thức tương đương trình độ trung cấp lý luận chính\ntrị.\n\nTrong đó, đối tượng dự tuyển đào tạo đại học chính quy tuyển mới đối với người\nđã tốt nghiệp trung học phổ thông và tương đương được quy định cụ thể tại\nKhoản 1 Điều 7 của dự thảo, gồm:\n\na) Cán bộ Công an;\n\nb) Chiến sĩ nghĩa vụ trong CAND có thời gian công tác từ 12 tháng trở lên;\ncông dân hoàn thành thực hiện nghĩa vụ tham gia CAND hoặc hoàn thành thực hiện\nnghĩa vụ quân sự trong Quân đội nhân dân trong thời gia

In [70]:
for doc in results:
    print(doc.page_content)

– Dự tuyển đào tạo liên thông giữa trình độ trung cấp, trình độ cao đẳng với
trình độ đại học;

– Dự tuyển đào tạo đại học cấp bằng đại học thứ hai (văn bằng 2);

– Dự tuyển đào tạo trung cấp chính quy tuyển mới;

– Dự tuyển đào tạo trung cấp chính quy cho cán bộ trong biên chế CAND;

– Dự tuyển giáo dục trung học phổ thông (đào tạo văn hóa nhỏ tuổi);

– Dự tuyển đào tạo cao cấp lý luận chính trị và hoàn chỉnh kiến thức cao cấp
lý luận chính trị;

– Dự tuyển hoàn thiện kiến thức tương đương trình độ trung cấp lý luận chính
trị.

Trong đó, đối tượng dự tuyển đào tạo đại học chính quy tuyển mới đối với người
đã tốt nghiệp trung học phổ thông và tương đương được quy định cụ thể tại
Khoản 1 Điều 7 của dự thảo, gồm:

a) Cán bộ Công an;

b) Chiến sĩ nghĩa vụ trong CAND có thời gian công tác từ 12 tháng trở lên;
công dân hoàn thành thực hiện nghĩa vụ tham gia CAND hoặc hoàn thành thực hiện
nghĩa vụ quân sự trong Quân đội nhân dân trong thời gian không quá 12 tháng kể
từ ngày có quyết định xuấ

In [8]:
embeddings = HuggingFaceEmbeddings(model_name='nlplabtdtu/sbert-70M-cased')

db_hybrid = ElasticsearchStore.from_documents(
    docs, 
    embeddings, 
    es_url="http://localhost:9200", 
    index_name="edu_hybrid",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True,
    )
)